# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

  **Чтобы выполнить поставленную задачу нам необходимо будет проделать несколько подготовительный действий и непосредственно проанализировать возможную прибыль в каждом из регонов. Дествовать будем следующим образом:**
- импортируем необходмые для работы библиотеки
- загрузим файлы с данными и изучим их размер, типы данных, наличие пропусков и коррелирующих значений
- выделим из датасетов признаки по которым модель будет обучаться и целевые показатели, котрые модель будет предсказывать
- разобъем данные на обучающуюю и валидационную выборки для обучения модели 
- проведем масштабирование признаков для лучшего обучения модели
- обучим модель линейной регрессии и предскажем объем сырья на каждой их скважин
- на основании предсказаний получим средний запас сырья в каждом из регионов и оценим работу модели с помощью метрики RMSE, которая покажет насколько модель ошибается относительно целевых показателей
- расчитаем достаточный объем сырья для безубыточной разработки сквадин и сравним его со средними запасми сырья в каждом из регионов
- выберем 200 скважин с самыми высокими предсказанными значениями, найдем общую сумму целевых показателей по этим скважинам и расчитаем прибыль для полученного обхема сырья
- с помощью техники Bootstrap найдем среднюю прибыль 95% доверительный интервал и процент риска убытков
- на основании полученных данных предложим регион для разработки с наименьшим риском убытков

## Загрузка и подготовка данных

In [2]:
# импортируем беблиотеки и необходимые инструменты
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState

In [3]:
# откроем файлы с данными и изучим их
region_0 = pd.read_csv('/datasets/geo_data_0.csv')
region_1 = pd.read_csv('/datasets/geo_data_1.csv')
region_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [4]:
region_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
region_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
region_0.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [7]:
region_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
region_1.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


Во втором исследуемом регионе корреляция Пирсона между признаком f2 и целевым показателем практически еденица, обученная нами модель будет приедсказывать целевые показатели в этом регионе точнее всего



In [9]:
region_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
region_2.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


*В каждом из прдоставленных нам датасетов содержатся по 10 000 строк с данными все столбцы кроме id содержат числовые значения, именно то что нужно для обучения нашей можели линейной регрессии.*

*Пропуски в данных отстутсвкют, датасеты одинкавы по размерам, сильнокоррелирующих признаков в данных нет, единственно в данных по региону 1 есть признак от значения котрого напрямую зависит целевой показатель количества сырья*

## Обучение и проверка модели

In [11]:
# разобъем данные на признаки и целевой показатель для всех регионов
features_0 = region_0.drop(['product', 'id'], axis = 1)
target_0 = region_0['product']

In [12]:
features_1 = region_1.drop(['product', 'id'], axis = 1)
target_1 = region_1['product']

In [13]:
features_2 = region_2.drop(['product', 'id'], axis = 1)
target_2 = region_2['product']

In [15]:
# разобъем полученные датасеты на обучающую и валидационные выборки
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0, target_0, 
                                                                                      test_size = 0.25, random_state = 12345)

In [16]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, 
                                                                                      test_size = 0.25, random_state = 12345)

In [17]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, 
                                                                                      test_size = 0.25, random_state = 12345)

In [23]:
# масштабируем признаки
scaler = StandardScaler()
scaler.fit(features_train_0) 
features_train_0 = scaler.transform(features_train_0)
features_valid_0 = scaler.transform(features_valid_0)

features_train_1 = scaler.transform(features_train_1)
features_valid_1 = scaler.transform(features_valid_1)

features_train_2 = scaler.transform(features_train_2)
features_valid_2 = scaler.transform(features_valid_2)

In [25]:
# обучим модель линейной регрессии, именно она нам подходит потому что остальные недостаточно предсказуемые
model = LinearRegression()
model.fit(features_train_0, target_train_0)
predicted_valid_0 = model.predict(features_valid_0)
mse = mean_squared_error(target_valid_0, predicted_valid_0)
mean_predicted_valid_0 = predicted_valid_0.sum()/len(predicted_valid_0)
print('RMSE региона 0:', mse**0.5)
print('Средний запас предсказанного сырья региона 0:', mean_predicted_valid_0 )

RMSE региона 0: 37.5794217150813
Средний запас предсказанного сырья региона 0: 92.59256778438035


In [28]:
model.fit(features_train_1, target_train_1)
predicted_valid_1 = model.predict(features_valid_1)
mse = mean_squared_error(target_valid_1, predicted_valid_1)
mean_predicted_valid_1 = predicted_valid_1.sum()/len(predicted_valid_1)
print('RMSE региона 1:', mse**0.5)
print('Средний запас предсказанного сырья региона 1:', mean_predicted_valid_1 )

RMSE региона 1: 0.893099286775617
Средний запас предсказанного сырья региона 1: 68.728546895446


In [31]:
model.fit(features_train_2, target_train_2)
predicted_valid_2 = model.predict(features_valid_2)
mse = mean_squared_error(target_valid_2, predicted_valid_2)
mean_predicted_valid_2 = predicted_valid_2.sum()/len(predicted_valid_2)
print('RMSE региона 2:', mse**0.5)
print('Средний запас предсказанного сырья региона 2:', mean_predicted_valid_2 )

RMSE региона 2: 40.02970873393434
Средний запас предсказанного сырья региона 2: 94.96504596800489


*Самый высокий запас предсказаного запаса сырья в регионе 2, в регионе 0 самый низкий показатель метрики RMSE то есть модель по данному региону наиболе точно предсказала запасы сырья. В регионе 1 и самый маленький предсказанный запась и самая низкая точность предсказания, несмотря на то что именно в этих данных один из признаков сильно скоррелиорван с целевым показателем*

## Подготовка к расчету прибыли

In [32]:
#Все ключевые значения для расчётов сохраним в отдельных переменных
WELL = 500 # количество исследуемых скважин
BEST_WELL = 200 # количество скважин выбранных по наилучшим предсказанным показателям
BUDGET = 10000000000 # бюджет на разработку 200 скважин в одном регионе
INCOME_PRODUCT = 450000 # доход с еденицы продукта(1 тыс баррелей)


In [33]:
# Найдем достаточный объем продукта для безубыточной разработки в регионах
sufficient_volume = BUDGET / BEST_WELL / INCOME_PRODUCT
print('Доставточный объем продукта для безубыточной разработки:', sufficient_volume)

Доставточный объем продукта для безубыточной разработки: 111.11111111111111


In [34]:
list_pred = [mean_predicted_valid_0, mean_predicted_valid_1, mean_predicted_valid_2]
for i in list_pred:
    if i > sufficient_volume:
        print('Средний запас сырья в регионе достаточен для безубыточной разработки')
    else:
        print(f'Средний запас сырья в регионе недостаточен для безубыточной разработки')

Средний запас сырья в регионе недостаточен для безубыточной разработки
Средний запас сырья в регионе недостаточен для безубыточной разработки
Средний запас сырья в регионе недостаточен для безубыточной разработки


*если рассматривать средний запас сырья по всем изучаемым точкам то во всех регионах этот показатель меньше чем минимально необходимый для безубыточной разработки, для более детального изучения отсеим точки с минимальным запасом сырья и изучим тольк 200 самых лучших скважин*

## Подсчет прибыли

In [36]:
# приведем к единому показателю индексы целевого показателя и полученных предсказаний?
# предсказания приведем к типу Series
target_valid_0 = target_valid_0.reset_index(drop=True)
predicted_valid_0 = pd.Series(predicted_valid_0)
target_valid_1 = target_valid_1.reset_index(drop=True)
predicted_valid_1 = pd.Series(predicted_valid_1)
target_valid_2 = target_valid_2.reset_index(drop=True)
predicted_valid_2 = pd.Series(predicted_valid_2)

In [37]:
#напишем функцию которая выберет 200 скважин с макимальными предсказаниям и подсчитает прибыль по целевому показателю
def profit(target, predictions, BEST_WELL):
    predictions_sorted = predictions.sort_values(ascending=False)
    best_selected = target[predictions_sorted.index][:BEST_WELL]
    income = best_selected.sum() * INCOME_PRODUCT - BUDGET
    return income

In [40]:
#подсчитаем прибыль в каждом регионе
print('Прибыль в регионе 0 составляет:', profit(target_valid_0, predicted_valid_0, BEST_WELL)/ 10**9, 'млрд.р.')
print('Прибыль в регионе 1 составляет:', profit(target_valid_1, predicted_valid_1, BEST_WELL)/ 10**9, 'млрд.р.')
print('Прибыль в регионе 2 составляет:', profit(target_valid_2, predicted_valid_2, BEST_WELL)/ 10**9, 'млрд.р.')

Прибыль в регионе 0 составляет: 3.3208260431398506 млрд.р.
Прибыль в регионе 1 составляет: 2.4150866966815108 млрд.р.
Прибыль в регионе 2 составляет: 2.7103499635998327 млрд.р.


*После исследования 200 лучших скважин в каждом из регионов на основе предсказанных данных, получили что наибольшая возможная прибыль будет получена в регионе 0, самая низкая брибыль в регионе 1, но помимо предсказанной возможной прибыли нам необходимо выяснить насколько велик риск убытков*

## Риски и прибыль для каждого региона

###  Применим технику Bootstrap с 1000 выборок.

In [42]:
state = np.random.RandomState(12345)

In [46]:
# Напишем фунцию которая расчитает нам среднюю прибыль, найдет доверительный интервал и покажет размер риска убытков
def bootstrap(target, predictions):
    values = []
    for i in range(1000):
        target_subsample = target.sample(n = 500, replace=True, random_state=state)
        predictions_subsumple = predictions[target_subsample.index]
        values.append(profit(target_subsample, predictions_subsumple, BEST_WELL))
        
    values = pd.Series(values)
    values_mean = int(values.mean())
    lower = int(values.quantile(q=0.025))
    upper = int(values.quantile(q=0.975))
    risk = int(len(values[values < 0]) / len(values) * 100)
    return values_mean, lower, upper, risk

### Посчитаем для каждого из регионов среднюю прибыль, 95-й доверительный интервал и риск убытков


In [47]:
values_mean_region_0, lower_region_0, upper_Region_0, risk__region_0  = (
    bootstrap(target_valid_0, predicted_valid_0)
)
print(f'Средняя прибыль 200 лучших месторождений региона 0: {values_mean_region_0}')
print(f'Нижняя граница доверительного интервала региона_0: {lower_region_0}')
print(f'Верхняя граница доверительного интервала региона 0: {upper_Region_0}')
print(f'Риск убытков региона 0: {risk__region_0} %' )


Средняя прибыль 200 лучших месторождений региона 0: 421563306
Нижняя граница доверительного интервала региона_0: -119499349
Верхняя граница доверительного интервала региона 0: 947039988
Риск убытков региона 0: 6 %


In [52]:
values_mean_region_1, lower_region_1, upper_Region_1, risk__region_1  = (
    bootstrap(target_valid_1, predicted_valid_1)
)
print(f'Средняя прибыль 200 лучших месторождений региона 1: {values_mean_region_1}')
print(f'Нижняя граница доверительного интервала региона_1: {lower_region_1}')
print(f'Верхняя граница доверительного интервала региона 1: {upper_Region_1}')
print(f'Риск убытков региона 1: {risk__region_1} %' )

Средняя прибыль 200 лучших месторождений региона 1: 500458025
Нижняя граница доверительного интервала региона_1: 90252863
Верхняя граница доверительного интервала региона 1: 925784776
Риск убытков региона 1: 1 %


In [55]:
values_mean_region_2, lower_region_2, upper_Region_2, risk__region_2  = (
    bootstrap(target_valid_2, predicted_valid_2)
)
print(f'Средняя прибыль 200 лучших месторождений региона 2: {values_mean_region_2}')
print(f'Нижняя граница доверительного интервала региона_2: {lower_region_2}')
print(f'Верхняя граница доверительного интервала региона 2: {upper_Region_2}')
print(f'Риск убытков региона 0: {risk__region_2} %' )

Средняя прибыль 200 лучших месторождений региона 2: 426072856
Нижняя граница доверительного интервала региона_2: -142980564
Верхняя граница доверительного интервала региона 2: 950219453
Риск убытков региона 0: 7 %


***Мы провели исследование по определения самого перспективного региона для дальнейшей разработки, изуили исходные данные, обучили модель для предсказания количесвта сырья в каждой из скважин, сравнили предсказания с целевыми показателями, расчитали тьочку безубыточности разработки, отобрали 200 скважин в каждом из регионов с самыми высокими запасами сырья, расчитали их среднюю прогнозируемую прибыль а так же вычислили процент риска убытков.***

***На основание всей проделанной выше работы, а так же условий которые были нам даны о том что риск убытков от разработки не должен превышать 2,5% мы пришли к выводу что сав=мым перспективным для разработки является регион 1 у него самый высокий показатель средней прибыли 200 самых лучших точек и риск аозникновения убытков от разработки всего одни процент.***
